In [ ]:
import numpy as np
import pandas as pd
import duckdb

In [ ]:
con = duckdb.connect(database = ":memory:")

In [ ]:
con.sql("CREATE OR REPLACE TABLE train AS SELECT * FROM read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')")

In [ ]:
# Number of rows
con.sql("SELECT count(*) FROM train")

In [ ]:
con.sql("SUMMARIZE train; ").df()

lab_id - The pseudonym of the lab that provided the data. The CalMS21, CRIM13, and MABe22 datasets are copies of publicly available datasets provided as additional training data. Some tracking files in CalMS21 set are largely duplicated-- these were annotated by multiple individuals for different sets of behaviors.

In [ ]:
# lab_id
con.sql("SELECT DISTINCT lab_id FROM train")

In [ ]:
con.sql("SELECT lab_id, COUNT(*) as values FROM train GROUP BY lab_id ORDER BY values DESC;")

video_id - A unique identifier for the video.

In [ ]:
con.sql("SELECT COUNT(video_id) FROM train;")

In [ ]:
print(con.sql("SELECT DISTINCT mouse1_strain from train ORDER BY mouse1_strain"))
print(con.sql("SELECT DISTINCT mouse2_strain from train ORDER BY mouse2_strain"))
print(con.sql("SELECT DISTINCT mouse3_strain from train ORDER BY mouse3_strain"))
print(con.sql("SELECT DISTINCT mouse4_strain from train ORDER BY mouse4_strain"))

In [ ]:
con.sql("SELECT mouse1_strain, count(*) AS counts FROM train GROUP BY mouse1_strain ORDER BY counts DESC")

### C57Bl/6J      
![](https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcTyCsmJnNL5EMT7podASllF5YX76lOmio-XqKstClgCe8MPVK0RKTEe37-yO5LoUMS_XPWF0FkjSBCgQJLIiHQSHj9HPJyeSzBiLWdYdg)



### BTBR
![BTBR](https://media.jax.org/m/57190339fb1108b2/original/002282.png)


###  C57Bl/6N  
![](https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQncuZ1zz9D6Mvg8fQqGkrqN2aebUKL_gFpW0DGIWxqTQSZP7bVcDPS9AS1jUXW2MeHBAP-mBZR1o7eeyg6-uRneAYCAICc5brBK4YxsdI)

### CD1       
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfwBnnnC46c39disTjb2PA0pzNRnPUxom4oQ&s)



###  C57Bl/6J x Ai148 
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRrrFDFTUWNwKf7zppGqCwgQ7JTolG0J_fTNg&s)

###  CD-1 (ICR)       
![](https://www.inotiv.com/hs-fs/hubfs/resources/model-images/ICR.png?width=350&height=181&name=ICR.png)

### 129/SvEvTac
![129/SvEvTac](https://assets.taconic.com/is/image/taconic/129s6-inbred-mouse-model-1?ts=1695185333723&dpr=off)

### CFW    
![](https://criver.widen.net/content/kzrrkzsckz/jpeg/RM-001459-white-mouse.jpeg?position=c&color=ffffffff&quality=80&u=dxbf0u)

### BALB/c
![](https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQIZj_eGKVrLhNpGjfClEdVEgCYzgNGPWJc4jX7-JhClFXI-Igb3Ov7TDfu0N2Vsmvmyg5ITH5WMe1xhgY6c3IvkyPklIuRn8M4LNBZMZk)

frames per second

In [ ]:
con.sql("SELECT frames_per_second, COUNT(*) AS counts FROM train GROUP BY frames_per_second ORDER BY counts DESC")

In [ ]:
con.sql("SELECT video_duration_sec, COUNT(*) AS counts FROM train GROUP BY video_duration_sec ORDER BY counts DESC")

arena

In [ ]:
con.sql("SELECT arena_shape, COUNT(*) AS counts  FROM train GROUP BY arena_shape ORDER BY counts DESC")

In [ ]:
con.sql("SELECT arena_type, COUNT(*) AS counts  FROM train GROUP BY arena_type ORDER BY counts DESC")

In [ ]:
con.sql("SELECT body_parts_tracked, COUNT(*) AS counts  FROM train GROUP BY body_parts_tracked ORDER BY counts DESC").df()

Tracking Method

### **Custom HRNet**
This refers to a modified version of the **High-Resolution Network (HRNet)**, a deep learning model originally designed for human pose estimation. It's adapted for tracking specific keypoints on animals or objects by maintaining high-resolution feature maps throughout the network, which allows for very precise spatial localization.

***

### **MARS**
**MARS (Mask-based Automatic Rater)** is a machine learning method specifically developed for tracking the poses of mice in a social context. It uses computer vision to automatically identify and follow multiple body parts of interacting animals, which helps researchers analyze social behavior without manual annotation.

***

### **DeepLabCut**
**DeepLabCut** is a popular open-source toolbox that enables markerless pose estimation of any animal or object. It leverages a pre-trained deep neural network (ResNet) and requires only a small number of labeled images to train a custom tracker for specific keypoints, making it highly versatile for scientific research.

***

### **SLEAP**
**SLEAP (Social Leap Estimates of Animal Poses)** is a deep learning framework for multi-animal pose tracking that is optimized for speed and accuracy, even in complex scenes with many interacting individuals. It can work with a single camera and is designed to handle social interactions where animals might block or touch each other.

In [ ]:
con.sql("SELECT tracking_method, COUNT(*) AS counts  FROM train GROUP BY tracking_method ORDER BY counts DESC")

The tracking methods listed (custom HRnet, MARS, DeepLabCut, SLEAP) are all computer vision-based techniques. They analyze video footage to estimate the pose and track the body parts of mice, typically without the need for physical sensors attached to the animals. This allows for the study of free-moving mice in their environment.

In [ ]:
con.sql("CREATE OR REPLACE TABLE test AS SELECT * FROM read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')")

In [ ]:
con.sql("SUMMARIZE test").df()

Create train_tracking view as we are just looking at the data, we can create external table to save memory
> add lab_id and video_id as features

In [ ]:
con.sql("""
CREATE OR REPLACE VIEW train_tracking AS 
SELECT split_part(filename, '/', 6) AS lab_id,
       replace(regexp_extract(filename, '([^/]+)$', 1), '.parquet', '') AS video_id,
       * EXCLUDE filename,
FROM read_parquet(
    '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/**/*.parquet',
    filename=true,
    hive_partitioning=false
);
""")

In [ ]:
con.sql("DESCRIBE train_tracking;").df()

In [ ]:
con.sql("SELECT * FROM train_tracking LIMIT 5;").df()

In [ ]:
con.sql("SELECT MAX(video_frame) AS total_frames FROM train_tracking WHERE video_id=1212811043;").df()

Validating one video for total frames

In [ ]:
con.sql("""
SELECT  frames_per_second, 
        video_duration_sec, 
        frames_per_second*video_duration_sec AS total_frames
FROM train
WHERE video_id=1212811043;
""").df()

train annotation

In [ ]:
con.sql("""
CREATE OR REPLACE VIEW train_annotation AS 
SELECT split_part(filename, '/', 6) AS lab_id,
       replace(regexp_extract(filename, '([^/]+)$', 1), '.parquet', '') AS video_id,
       * EXCLUDE filename,
FROM read_parquet(
    '/kaggle/input/MABe-mouse-behavior-detection/train_annotation/**/*.parquet',
    filename=true,
    hive_partitioning=false
);
""")

In [ ]:
con.sql("DESCRIBE train_annotation;")

In [ ]:
con.sql("SELECT * FROM train_annotation LIMIT 5;").df()

To be continued...